In [ ]:
#pip install pyspark

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Occupancy Estimation") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/24 22:04:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv('Occupancy_Estimation.csv', header=True, inferSchema=True)

In [ ]:
# Print the schema to understand the data types
df.printSchema()

# Summary statistics for numerical columns
df.describe().show()

#Features and Target

 - Target is predict Room Ocuppancy which is(Room_Occupancy_Count column)
 - All the sensors readings would be the features to be analysed

In [5]:
def create_sequences(X, y, time_steps=60):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        # Make sure not to exceed the bounds
        end_ix = i + time_steps
        if end_ix > len(X):
            break  # Break the loop if the end index exceeds the dataset size
        seq_X = X[i:end_ix]
        seq_y = y[end_ix - 1]  # Target is the last value in the sequence
        Xs.append(seq_X)
        ys.append(seq_y)
    return np.array(Xs), np.array(ys)


In [9]:
# Converting X and y inputs into NumPy arrays before passing them to the function 
# as the function uses array slicing which is more straightforward with NumPy arrays than
# with pandas Series or DataFrames.

# X_train_seq, y_train_seq = create_sequences(X_train.values, y_train.values, time_steps)
# X_test_seq, y_test_seq = create_sequences(X_test.values, y_test.values, time_steps)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [15]:
# Loading the dataset
df1 = pd.read_csv('Occupancy_Estimation.csv')

# Exclude 'Date', 'Time', and the target column for features
features = df1.drop(['Date', 'Time', 'Room_Occupancy_Count'], axis=1)
target = df1['Room_Occupancy_Count']

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler.fit_transform(features)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.3, random_state=42)

# Reshape data for LSTM [samples, time steps, features]
# This function will convert the data into sequences
def create_sequences(X, y, time_steps=60):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 60
X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, time_steps)


KeyError: 61

In [ ]:
# Excluding 'Date', 'Time', and 'Room_Occupancy_Count' - Those are not need features
# I could have set just 16 as my Data set has 19 features but as good practice 
# If I get a large data set no need to count the number of features with this code in case of recycling code.

number_of_features = len(df.columns) - 3  # Adjusted for Date, Time, and target column
time_steps = 60
input_shape = (time_steps, number_of_features)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Example model structure
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(input_shape)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')


In [7]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(8103, 16) (8103,) (2026, 16) (2026,)


In [14]:
def create_sequences(X, y, time_steps=60):
    Xs, ys = [], []
    max_start_index = len(X) - time_steps
    if max_start_index < 0:
        raise ValueError("The dataset is too small for the number of time steps specified.")
    for i in range(max_start_index + 1):  # Ensure we do not go beyond the dataset
        seq_X = X[i:(i + time_steps)]
        seq_y = y[i + time_steps - 1]  # Adjusted to ensure it's within bounds
        Xs.append(seq_X)
        ys.append(seq_y)
    return np.array(Xs), np.array(ys)

In [13]:
print(df1.columns)  # For DataFrames
print(df1.index)    # For checking indices


Index(['Date', 'Time', 'S1_Temp', 'S2_Temp', 'S3_Temp', 'S4_Temp', 'S1_Light',
       'S2_Light', 'S3_Light', 'S4_Light', 'S1_Sound', 'S2_Sound', 'S3_Sound',
       'S4_Sound', 'S5_CO2', 'S5_CO2_Slope', 'S6_PIR', 'S7_PIR',
       'Room_Occupancy_Count'],
      dtype='object')
RangeIndex(start=0, stop=10129, step=1)
